# Analyze scRNS-seq jointly with CRISPRa screening result

In [ ]:
%config InlineBackend.figure_formats = ['svg']
import lamindb as ln
import scanpy as sc

ln.track("F4L3oC6QsZvQ", project="Schmidt22")

Load hits from phenotypic screen.

In [ ]:
artifact_hits = ln.Artifact.get(key="schmidt22/gws-crispr-ifng-hits.parquet")
artifact_hits.describe()
screen_hits = artifact_hits.load()
screen_hits.head()

Load the scRNA-seq data.

In [ ]:
artifact_scrna = ln.Artifact.get(
    key="schmidt22_perturbseq/schmidt22_perturbseq_ensembl_gene_ids.h5ad"
)
artifact_scrna.describe()
adata = artifact_scrna.load()
adata

Investigate with which transcriptional states the phenotypic correlate.

In [ ]:
sc.tl.score_genes(
    adata,
    adata.var_names.intersection(
        screen_hits["target_gene_ensembl_id"],
    ).tolist(),
    score_name="enrichment_score_for_screen_results",
)

Visualize the results.

In [ ]:
sc.pl.matrixplot(
    adata,
    groupby="scrna_cell_state",
    var_names=["enrichment_score_for_screen_results"],
    swap_axes=True,
)

Also visualize with a UMAP.

In [ ]:
sc.pl.umap(adata, color=["enrichment_score_for_screen_results", "scrna_cell_state"])

Check the output matrix.

In [ ]:
adata

Save the output.

In [ ]:
artifact_result = ln.Artifact.from_anndata(
    adata,
    key="schmidt22/analyzed_data.h5ad",
    schema="ensembl_gene_ids_and_valid_features_in_obs",
    features={
        "experiment": artifact_scrna.features.get_values()["experiment"],
        "biosample": artifact_scrna.features.get_values()["biosample"],
    },
).save()
artifact_result.describe()

In [ ]:
artifact_result.view_lineage()

In [ ]:
ln.finish()